In [5]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from pydantic import BaseModel, Field
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-0125")

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class CompanyOverviewToolArgsSchema(BaseModel):
    symbol: str = Field(description="Stock symbol of the company.Example: AAPL, TSLA")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for")


class CompanyStackPerformanceTool(BaseTool):
    name: Type[str] = "CompanyStackPerformance"
    description: Type[
        str
    ] = """
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewToolArgsSchema] = CompanyOverviewToolArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


class CompanyIncomeStatementTool(BaseTool):
    name: Type[str] = "CompanyIncomeStatement"
    description: Type[
        str
    ] = """
    Use this to get an income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewToolArgsSchema] = CompanyOverviewToolArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()["annualReports"]


class CompanyOverviewTool(BaseTool):
    name: Type[str] = "CompanyOverview"
    description: Type[
        str
    ] = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewToolArgsSchema] = CompanyOverviewToolArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


class StockMarketSymbolSearchTool(BaseTool):
    name: Type[str] = "StockMarketSymbolSearch"
    description: Type[
        str
    ] = """
    Use this tool to find the stock market symbool for a company.
    It takes a query as an argument.
    Example query: Stock Market Symbol for Apple Company.
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = (
        StockMarketSymbolSearchToolArgsSchema
    )

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStackPerformanceTool(),
    ],
)

prompt = "Give me information on Cloudflare's stock, considering its financials, income statement and stock performance and help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearch` with `{'query': 'Stock Market Symbol for Cloudflare Company'}`


Real-time Cloudflare stock price (NYSE: ... Market Cap $38.67 Bil. Volume/Avg ... 0.00%. Dividend Yield (Forward) 0.00%. Total Yield — Company Profile . Cloudflare is a software company based in ... About NET. Cloudflare, Inc. operates as a cloud services provider that delivers a range of services to businesses worldwide. The company provides an integrated cloud-based security solution to secure a range of combination of platforms, including public cloud, private cloud, on-premise, software-as-a-service applications, and IoT devices; and website and application security products ... Get Cloudflare Inc (NET) real-time stock quotes, news, price and financial information from Reuters to inform your trading and investments Specifically, they have bought $0.00 in company stock and sold $61,334,450.00 in company stock. Percentage Held by Insiders. 12.8

{'input': "Give me information on Cloudflare's stock, considering its financials, income statement and stock performance and help me analyze if it's a potential good investment.",
 'output': "### Cloudflare Inc. Stock Analysis:\n\n#### Company Overview:\n- **Symbol:** NET\n- **Description:** Cloudflare Inc. operates a cloud platform that offers a range of network services to companies worldwide.\n- **Market Capitalization:** $38.29 billion\n- **Revenue (TTM):** $1.57 billion\n- **Gross Profit (TTM):** $742.63 million\n- **EPS (TTM):** -$0.27\n- **Profit Margin:** -5.97%\n- **Operating Margin (TTM):** -7.16%\n- **Return on Assets (TTM):** -3.27%\n- **Return on Equity (TTM):** -11.2%\n- **Analyst Target Price:** $97.82\n- **52-Week High:** $119.42\n- **52-Week Low:** $66.24\n\n#### Income Statement:\n- **Latest Fiscal Year End:** December 2023\n- **Revenue:** $1.3 billion\n- **Gross Profit:** $989.74 million\n- **Operating Income:** -$185.48 million\n- **Net Income:** -$183.95 million\n\